# subband filtering
32個のバンドパスフィルターによるフィルタバンクで，信号を32等分されたサブバンド信号に分割する．
各フィルタは512点インパルス応答であり，規格で決められたベースバンドプロトタイプから作られる．

In [ ]:

baseband_filter = filter_coeffs()

ベースバンドフィルタのプロトタイプのインパルス応答と，振幅スペクトラムは以下の図のようになっている．サンプリング周波数が44.1kHzで，カットオフ周波数がfs/64．
見ればわかるように，幅fs/32なので，32個の等分されたバンドバスで全帯域をカバーできる．

In [ ]:
fig = plt.figure()

このプロトタイプを変調することで，フィルタバンクを作る．各フィルタは，$\pi/64$の奇数倍に中心化する．つまりプロトタイプにその周波数のコサインを乗算する．

In [ ]:
filterbank = np.zeros((N_SUBBANDS,FRAME_SIZE), dtype='float32')

for sb in range(N_SUBBANDS):
    for n in range(FRAME_SIZE):
        filterbank[sb,n] = baseband_filter[n] * np.cos((2*sb+1)*(n-16)*np.pi/64)

fig2 = plt.figure()
ax21 = fig2.add_subplot(1,1,1)


## サンプルをフィルタリング

フィルタバンクを作り終えて，32個のインパルス応答ができた．実際にフィルタリングする．

MPEG1標準規格では，フレーム長を512点としている．フィルタリングの結果となるサブバンド信号の点数は，信号の点数より32倍多い．よって，厳密に32の階乗によってサブサンプルされる．

これは完全に有効な実装であるけど，それは，それぞれの新しい入力信号の512点の32個のフィルタごとに出力を計算して，フィルタの31/32の連続した出力をダウンサンプリングで無視することを意味する．
これは明らかにとても富豪な方法だから，サブバンドフィルタリングは，少し整頓した異なる方法でおこなう．
各点に対してフィルタリングを行うときに1点シフトをする代わりに，入力信号の32点シフトを使う．そうすると，フィルタリングは，32点それぞれにおこなわれ，サブサンプリングが必要なくなる．詳しくは[２]を参照

この実装は必要な操作を最小限に抑えるという点で最適な実装ではないけれど，ISO規格は、ウィンドウ処理、部分計算、修正離散コサイン変換で構成されるこの実装を提供します。 詳細は[2]にもあります。 速度の面でそれほど大きな改善をもたらさず、さらに重要なことに、このコースの範囲内で理解するのがはるかに難しいため、これを使用するつもりはありません。

では、サブバンドフィルタリングのコードを見てみましょう。 range（FRAMES_PER_BLOCK）(=12)のループがあることがわかります。その理由は、MPEG-1レイヤー1では、出力ファイルに書き込まれるときに、後続の12フレームが処理されてブロックとして一緒にまとめられることを指定しているためです。

In [ ]:
subband_samples = np.zeros((params.nch, N_SUBBANDS, FRAMES_PER_BLOCK), dtype='float32') 

# Processing is done for 12 subsequent frames
for frm in range(FRAMES_PER_BLOCK):
      # Read in next 32 samples
      samples_read = input_buffer.read_samples(SHIFT_SIZE)
      # Zero-pad the signal if EOF is reached
      if samples_read < SHIFT_SIZE:
        for ch in range(params.nch):
          input_buffer.audio[ch].insert(np.zeros(SHIFT_SIZE - samples_read)) 
      # Filtering = dot product with reversed buffer      
      for ch in range(params.nch):   
        subband_samples[ch,:,frm] = np.dot(filterbank, input_buffer.audio[ch].reversed()) 